# Базовый парсер

Вытаскивает из latex-кода заголовки статей и их расположение в файлах.

Разбивка происходит в полуручном режиме, т.к. нет уверенности в формате заголовков.

В тексте ищутся слова, содержащие в своём составе заглавные буквы на русском и английском языках в отношении, большем или равным заданному (по умолчанию 0.66). Предполагается, что таким образом удаётся обнаруживать неправильно машиинно распознанный капс. Слова или цепочки слов, состоящие из одного строчного символа включаются в заголовок, если стоят между слов, определённых как часть заголовка. При этом, одиночные заглавные буквы, а также инициалы не воспринимаются как начало заголовка.

## Использование
- При удовлетворительном определении заголовка нажать `Enter` без дополнительного ввода.
- Если предложенное место заголовком не является ввести `"n"`
- При неправильном определении границ заголовка ввести два корректировочных числа для сдвига левой и правой границы.
  - ЗАМЕЧАНИЕ: сдвиг производится попробельно, т.е. двойной пробел будет распознан как слово нулевой длины.
  - ЗАМЕЧАНИЕ: границы отображаемого фрагмента текста будут передвинуты автоматически. Длины левой и правой границ в словах задаются в параметрах.
  - ПРИМЕРЫ:
    - `out: a [B C] d e f` -> `in: 0 2` -> `out: a [B C D E] f`
    - `out: a b c [D E] f` -> `in: 2 -1` -> `out: a [B C D] e f`
- Также возможен посимвольный сдвиг правой границы в случае "сращивания" заголовка статьи и её текста. Ввести одно число, начиная с точки.
  - ПРИМЕРЫ:
    - `out: a[BC]def` -> `in: .2` -> `out: a[BCDE]f`
    - `out: a[BCDE]f` -> `in: .-1` -> `out: a[BCD]ef`

В выводе в терминале переносы строк для удобства заменены на `"$"`

### Прочее
- Для определителя капса достуны исключения, которые никогда не будут рассматриваться, как потенциальные начала заголовков, см. опции. По умолчанию: первые 10 римских цифр и "МэВ".
- Использовать системный терминал для взаимодействия оказывается удобнее, чем использовать jupyter, поэтому можно скопировать ячейку с кодом в файл `scripter.py` и запускать его.
- При положительном определении заголовка файл дополняется немедленно, прервать процесс можно в любой момент, как и продолжить после -- итоговый файл будет дополяться, а не перезаписываться с нуля при новом запуске программы (главное не забыть предварительно удалить из конца файла дубликаты, если вы начинаете с той страницы, на которой закончили в прошлый раз, а не со следующей).

In [1]:
from os import walk
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import codecs


############################ VARS ################################
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./matphys/"
EXIT_FILE = "FMEv2.xml"
# First and last pages to be parsed
START_PAGE = 163
END_PAGE = 200
# How many words to display before and after a potential title
LEAD_WORDS = 5
AFT_WORDS = 5
# Look in the description
CAPS_QUOT = 0.51
EXCEPTIONS = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'МэВ']
##################################################################



class Article:
	start_title = 0
	end_title = 0
	filename = ''



# Write xml tree to file
def prettify(elem):
	# Pretty-printed XML string for the Element.
	rough_string = ET.tostring(elem, 'utf-8')
	reparsed = minidom.parseString(rough_string)
	return reparsed.toprettyxml(indent="  ")
def xml_write(root):
	with codecs.open(EXIT_DIR + EXIT_FILE, 'w', 'utf-8') as f:
		f.write(prettify(root))


# Get filenames needed
filenames_raw = next(walk(PAGES_DIR), (None, None, []))[2]  # [] if no file
filenames = []
for i in range(START_PAGE, END_PAGE + 1):
	for filename in filenames_raw:
		beginning = "rp-" + str(i) + "_"
		if filename[:len(beginning)] == beginning and filename[-4:] == ".mmd":
			filenames.append(filename)
						

# Check for existing xml
filenames_raw = next(walk(EXIT_DIR), (None, None, []))[2]  # [] if no file
if not(EXIT_FILE in filenames_raw):
	root = ET.Element('data')
	xml_write(root)


# Parse existing xml (string parsing is needed to avoid extra newlines appearing)
exit_string = ''
with codecs.open(EXIT_DIR + EXIT_FILE, 'r', 'utf-8') as f:
	for i in f.readlines():
		exit_string += i[:-1]
root = ET.fromstring(exit_string)
# Remove empty tails and texts
root.tail = None
root.text = None
for i in root:
	i.tail = None
	i.text = None
	for j in i:
		j.tail = None
		is_space = True
		for letter in j.text:
			is_space = False if letter != ' ' else is_space
		j.text = None if is_space else j.text
		for k in j:
			k.tail = None
			is_space = True
			for letter in k.text:
				is_space = False if letter != ' ' else is_space
			k.text = None if is_space else k.text


# Add article title and metadata to xml tree
def add_artice(elem, root, num):
	article = ET.SubElement(root, 'article', {'n':str(num)})
	title = ET.SubElement(article, 'title')
	title.text = file[elem.start_title+1:elem.end_title]
	title_meta = ET.SubElement(article, 'title-meta')
	title_file = ET.SubElement(title_meta, 'title-file')
	title_file.text = elem.filename
	title_start = ET.SubElement(title_meta, 'title-start')
	title_start.text = str(elem.start_title + 1)
	title_end = ET.SubElement(title_meta, 'title-end')
	title_end.text = str(elem.end_title)
	xml_write(root)


# Count number of alphabetic letters in word
def count_letters(word):
	num = 0
	for letter in word:
		num += 0 if re.match(r"[A-ZА-Яa-zа-я]", letter) == None else 1
	return num

# Check if word is written in CAPS
def check_caps(word):
	num = 0
	len_word = 0
	for letter in word:
		#num += 0 if re.match(r"[A-ZА-Я0-9]|[!#$%&'*+-.^_`|~:]", letter) == None else 1					# Too many symbols, math formulas are being detected
		len_word += 1 if re.match(r"[!#$%&'*+-.^_`|~:]", letter) == None else 0
		num += 0 if re.match(r"[A-ZА-Я]", letter) == None else 1
	return 0 if len_word == 0 or num / len_word < CAPS_QUOT or word in EXCEPTIONS else num				# Also exclude common roman numbers

# Check for initials like "I.E."
def check_initials(word):
	initials = True
	for i in range(len(word) - 1):
		type_1 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word[i]) == None else 1
		type_2 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word[i + 1]) == None else 1
		initials = False if type_1 and type_2 else initials
	return initials


# Find next ot prev word boundary (space / newline)
def prev_from(pos, file):
	pos = max(pos, 0)
	prev_space = file.rfind(' ', 0, pos)
	prev_nl = file.rfind('\n', 0, pos)
	prev_space = -1 if prev_space == -1 else prev_space
	prev_nl = -1 if prev_nl == -1 else prev_nl
	return max(prev_nl, prev_space)
def next_from(pos, file, end_replace = True):
	next_space = file.find(' ', pos + 1)
	next_nl = file.find('\n', pos + 1)
	if end_replace:
		next_space = len(file) if next_space == -1 else next_space
		next_nl = len(file) if next_nl == -1 else next_nl
	return max(next_nl, next_space) if next_space == -1 or next_nl == -1 else min(next_nl, next_space)


# Main loop
num = len(root) + 1
for filename in filenames:
	print()
	print("################################ " + filename + " ################################")
	with codecs.open(PAGES_DIR + filename, 'r', 'utf-8') as f:
		file = f.read()
	
	word_bound_l = -1
	word_bound_r = next_from(word_bound_l, file, end_replace=False)
	EOF_reached = False

	while not EOF_reached:
		if word_bound_r == -1:
			word_bound_r = len(file)
			EOF_reached = True


		if check_caps(file[word_bound_l+1:word_bound_r]) < 2 or check_initials(file[word_bound_l+1:word_bound_r]):
			word_bound_l = word_bound_r
			word_bound_r = next_from(word_bound_l, file, end_replace=False)
		
		else: # Possibly found a title
			# Left border of a title is already known
			start_title = word_bound_l

			# Define right border of a title
			defined_end = False
			end_title = word_bound_r
			while not defined_end:
				word_bound_l = word_bound_r
				word_bound_r = next_from(word_bound_l, file)

				if word_bound_l == len(file):
					defined_end = True
				elif not check_caps(file[word_bound_l+1:word_bound_r]) and count_letters(file[word_bound_l+1:word_bound_r]) < 2:
					pass
				elif check_caps(file[word_bound_l+1:word_bound_r]):
					end_title = word_bound_r
				else:
					defined_end = True

			next_title = False
			while not next_title:
				# Console output for further user actions
				segment_start = start_title
				segment_end = end_title
				for i in range(LEAD_WORDS):
					segment_start = prev_from(segment_start, file)
				for i in range(AFT_WORDS):
					segment_end = next_from(segment_end, file)
				
				out_str = file[segment_start+1:segment_end]

				# Format
				for i in range(len(out_str)):
					out_str = out_str[:i] + ('$' if out_str[i] == '\n' else out_str[i]) + out_str[i+1:]
				out_str = f"{num})\n" + out_str + '\n' + ' ' * (start_title - segment_start) + '^' * (end_title - start_title - 1)
				# Check for "section" in the string. This is referred to alphabetic tip at the bottom of the page
				"""if 'section' in out_str or 'title' in out_str:
					out_str += '     ############################### Title or section found! ###############################'""" # Not Used
				print(out_str)

				# User actions
				response = input()
				try:
					if response == '':
						# Add article
						article = Article()
						article.start_title = start_title
						article.end_title = end_title
						article.filename = filename
						add_artice(article, root, num)
						num += 1
						next_title = True
						print(f"Adding article, n=\"{num}\", title=\"{file[start_title+1:end_title]}\"\n\n")
					elif response == 'n' or response == 'т':
						# Do not add this one
						next_title = True
						print("Not an article, skipping\n\n")
					elif response[0] == '.':
						end_title += int(response[1:])
						print("Changing title right border\n\n")
					else:
						# Change title borders
						corrections = response.split(' ')
						corrections[0] = int(corrections[0])
						corrections[1] = int(corrections[1])
						if corrections[0] > 0:
							for i in range(abs(corrections[0])):
								start_title = prev_from(start_title, file)
						if corrections[0] < 0:
							for i in range(abs(corrections[0])):
								start_title = next_from(start_title, file)
						if corrections[1] < 0:
							for i in range(abs(corrections[1])):
								end_title = prev_from(end_title, file)
						if corrections[1] > 0:
							for i in range(abs(corrections[1])):
								end_title = next_from(end_title, file)
						print("Changing title borders\n\n")
				except:
					print("########## !!! Failed on input, try again !!! ##########\n\n")


# End reached
print('###########################################################################################')
print('Last requested page processd. Press "Enter" to close this window.')
response = input()


################################ rp-104_2023_07_08_c9e658ddccab043daaa4g.mmd ################################
17)
a_{l}^{-} a_{m}^{-},$\]$$104 ВТОРИЧНОЕ где$$\[$\begin{gathered}$H_{i
                             ^^^^^^^^^
Changing title borders




ValueError: invalid literal for int() with base 10: 'т'

# Исправление ошибок в заголовках

Состоит из двух частей: "составитель" и "заменитель".

Сначала формируется xml список всех заголовков с возможными автоматическими исправлениями (в формате было / стало):
1. замена латиницы на агалогичную кириллицу;
2. удаление обрамляющих знакоав препинания;
3. замена всех букв на заглавные (в том числе это избавляет дальнейшей необходимости исправлять имена);
4. слияние разорванных на отдельные буквы слов (если рядом оказываются несколько таких слов, то они оказываются слиты вместе).

После необходимо его просмотреть и исправить оставшиеся ошибкию

Затем запустить "заменитель", который заменит все заголовки на исправленныею

## Составитель:

In [2]:
from os import walk
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import codecs


############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
##################################################################



# Write xml tree to file
def prettify(elem):
	# Pretty-printed XML string for the Element.
	rough_string = ET.tostring(elem, 'utf-8')
	reparsed = minidom.parseString(rough_string)
	return reparsed.toprettyxml(indent="  ")
def xml_write(root):
	with codecs.open(WORK_DIR + CORRECTION_FILE, 'w', 'utf-8') as f:
		f.write(prettify(root))
						

# Check for existing xml
filenames_raw = next(walk(WORK_DIR), (None, None, []))[2]  # [] if no file
if not(INPUT_FILE in filenames_raw):
	root = ET.Element('data')
	xml_write(root)


# Parse input xml
exit_string = ''
with codecs.open(WORK_DIR + INPUT_FILE, 'r', 'utf-8') as f:
	for i in f.readlines():
		exit_string += i[:-1]
root = ET.fromstring(exit_string)


# Get all the titles into a dict
titles_dict = {}
for article in root:
	for elem in article:
		if elem.tag == 'title':
			titles_dict[elem.text] = elem.text


# Correct latin letters
letter_corr = {'A':'А', 'a':'а', 'B':'В', 'b':'Ь', 'E':'Е', 'e':'е', 'H':'Н', 'K':'К', 'M':'М', 'O':'О', 'P':'Р', 'p':'р', 'T':'Т', 'X':'Х', 'x':'x', 'y':'у', '6':'б'}
# Rarely seen
letter_corr['U'] = 'И'
letter_corr['r'] = 'г'
letter_corr['n'] = 'п'
for title in titles_dict.keys():
	title_new = titles_dict[title]
	for i in range(len(title_new)):
		if title_new[i] in letter_corr:
			title_new = title_new[:i] + letter_corr[title_new[i]] + title_new[i+1:]
	titles_dict[title] = title_new

# Remove bounding symbols
for title in titles_dict.keys():
	title_new = titles_dict[title]
	while re.match(r"[!#%&'*+-.^_`|~:]", title_new[0]) != None:
		title_new = title_new[1:]
	while re.match(r"[!#%&'*+-.^_`|~:]", title_new[-1]) != None:
		title_new = title_new[:-1]
	titles_dict[title] = title_new

# CAPS
for title in titles_dict.keys():
	titles_dict[title] = titles_dict[title].upper()

# Merge single-lettered words
for title in titles_dict.keys():
	title_new = titles_dict[title]
	title_new = ' ' + title_new + ' '
	for i in range(len(title_new) - 4):
		if (title_new[i] == ' ' or title_new[i] == '№') and title_new[i + 2] == ' ' and title_new[i + 4] == ' ':
			title_new = title_new[:i+2] + '№' + title_new[i+3:]
	i = 0
	while i < len(title_new):
		if title_new[i] == '№':
			title_new = title_new[:i] + title_new[i+1:]
			i = 0
		else:
			i += 1
	title_new = title_new[1:-1]
	titles_dict[title] = title_new


# Write corrections xml
root = ET.Element('data')
for i in titles_dict.items():
	pair = ET.SubElement(root, 'pair')
	title_old = ET.SubElement(pair, 'title_old')
	title_old.text = i[0]
	title_new = ET.SubElement(pair, 'title_new')
	title_new.text = i[1]
xml_write(root)

## Заменитель:

In [56]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import codecs


############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
EXIT_FILE = "FMEtitles.xml"
##################################################################



# Write xml tree to file
def prettify(elem):
	# Pretty-printed XML string for the Element.
	rough_string = ET.tostring(elem, 'utf-8')
	reparsed = minidom.parseString(rough_string)
	return reparsed.toprettyxml(indent="  ")
def xml_write(root):
	with codecs.open(WORK_DIR + EXIT_FILE, 'w', 'utf-8') as f:
		f.write(prettify(root))


# Parse input xml
exit_string = ''
with codecs.open(WORK_DIR + CORRECTION_FILE, 'r', 'utf-8') as f:
	for i in f.readlines():
		exit_string += i[:-1]
root = ET.fromstring(exit_string)


# Get all the corrections into a dict
titles_dict = {}
for pair in root:
	for elem in pair:
		if elem.tag == 'title_old':
			title_old = elem.text
		if elem.tag == 'title_new':
			title_new = elem.text
	titles_dict[title_old] = title_new


# Parse existing exit xml (string parsing is needed to avoid extra newlines appearing)
exit_string = ''
with codecs.open(WORK_DIR + INPUT_FILE, 'r', 'utf-8') as f:
	for i in f.readlines():
		exit_string += i[:-1]
root = ET.fromstring(exit_string)
# Remove empty tails and texts
root.tail = None
root.text = None
for i in root:
	i.tail = None
	i.text = None
	for j in i:
		j.tail = None
		is_space = True
		for letter in j.text:
			is_space = False if letter != ' ' else is_space
		j.text = None if is_space else j.text
		for k in j:
			k.tail = None
			is_space = True
			for letter in k.text:
				is_space = False if letter != ' ' else is_space
			k.text = None if is_space else k.text


# Replace titles
for article in root:
	for elem in article:
		if elem.tag == 'title':
			elem.text = titles_dict[elem.text]
xml_write(root)